# Converting words to floating point numbers and viceversa

In [1]:
import pandas as pd
import numpy as np

## First attempt

Given a string of words describing a floating point number, convert it into a floating point object.

In [2]:
def words_to_float_v1(text: str) -> float:
    # Mapping for number words
    num_map = {
        "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4", 
        "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9"
    }
    
    # Normalise input
    words = text.lower().strip().split()
    
    if "point" in words:
        point_index = words.index("point")
        int_part_words = words[:point_index]
        frac_part_words = words[point_index + 1:]
    else:
        int_part_words = words
        frac_part_words = []
    
    # Convert integer and fractional parts
    int_part = "".join(num_map[w] for w in int_part_words if w in num_map)
    frac_part = "".join(num_map[w] for w in frac_part_words if w in num_map)
    
    # Build final string and convert to float
    number_str = int_part if int_part else "0"
    if frac_part:
        number_str += "." + frac_part
    
    return float(number_str)

# Example usage:
print(words_to_float_v1("four point six one zero"))   # 4.610
print(words_to_float_v1("seven point two five"))      # 7.25
print(words_to_float_v1("nine"))                      # 9.0

4.61
7.25
9.0


## Second attempt

The previous function can be refactored into a most compact one.

In [3]:
def words_to_float_v2(text: str) -> float:
    num_map = dict(zip(
        ["zero","one","two","three","four","five","six","seven","eight","nine"],
        "0123456789"
    ))
    int_part, _, frac_part = text.lower().partition("point")
    return float(
        "".join(num_map.get(w, "") for w in int_part.split()) +
        ("." + "".join(num_map.get(w, "") for w in frac_part.split()) if frac_part else "")
    )

# Examples
print(words_to_float_v2("four point six one zero"))  # 4.610
print(words_to_float_v2("seven point two five"))     # 7.25
print(words_to_float_v2("nine"))                     # 9.0

4.61
7.25
9.0


## Third attempt

Now, we can use the index in the list to refactor the map definition.

In [4]:
def words_to_float_v3(text: str) -> float:
    num_map = {w: str(i) for i, w in enumerate(
              ["zero","one","two","three","four","five","six","seven","eight","nine"]
    )}
    int_part, _, frac_part = text.lower().partition("point")
    return float(
        "".join(num_map.get(w, "") for w in int_part.split()) +
        ("." + "".join(num_map.get(w, "") for w in frac_part.split()) if frac_part else "")
    )

# Examples
print(words_to_float_v3("four point six one zero"))  # 4.610
print(words_to_float_v3("seven point two five"))     # 7.25
print(words_to_float_v3("nine"))                     # 9.0

4.61
7.25
9.0


## The inverse

And, what about to the opposoite action?

In [5]:
def float_to_words(num: float) -> str:
    d = "zero one two three four five six seven eight nine".split()
    s = str(num)
    return " ".join((["minus"] if s.startswith("-") else (["plus"] if s.startswith("+") else [])) +
                    ["point" if c=="." else d[int(c)] for c in s.lstrip("+-") if c.isdigit() or c=="."])

# Examples
print(float_to_words(-4.61))   # "minus four point six one"
print(float_to_words(+7.25))   # "plus seven point two five"
print(float_to_words(9.0))     # "nine point zero"

minus four point six one
seven point two five
nine point zero


## The ultimate function

Can both approaches be combined into a one function?

In [6]:
def from_or_to_number(value: str | float) -> float | str:
    if value in [None, ""]:
        return None

    num_map = dict()
    if type(value) == str:
        num_map = { w:str(i) for i, w in enumerate(
                      ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
        )}
        i, _, f = value.lower().partition("point")
        return float("".join([ "minus" ] if i.startswith("-") else [ "plus" ] if i.startswith("+") else [] +
                             [ "".join(num_map.get(w, "") for w in i.split()) ] +
                             [ "." ] +
                             [ "".join(num_map.get(w, "") for w in f.split()) ]
                            )
        )
    elif type(value) in [ int, float ]:
        num_map = { str(i):w for i, w in enumerate(
                      ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
        )}
        s = str(value)
        return " ".join([ "minus"] if s.startswith("-") else ["plus"] if s.startswith("+") else [] +
                        [ "point" if c=="." else num_map[c] for c in s if c.isdigit()]
        )
    else:
        return None



# Examples
print(from_or_to_number("four point six one zero"))  # 4.610
print(from_or_to_number("seven point two five"))     # 7.25
print(from_or_to_number("nine"))                     # 9.0

print(from_or_to_number(4.610))                      # "four point six one zero"
print(from_or_to_number(7.25))                       # "seven point two five"
print(from_or_to_number(9.0))                        # "nine"

4.61
7.25
9.0
four six one
seven two five
nine zero


## Excercise

### Example dataset

In [7]:
df = pd.DataFrame({'values': np.random.uniform(-10, 10, 10)})
print(df)

     values
0  4.211163
1  0.063325
2 -2.348341
3 -4.194727
4  6.225549
5 -6.948032
6  0.697528
7  8.177460
8 -8.918164
9  8.836819
